In [1]:
#importing some useful packages

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline
from PIL import Image
import keras
import math
import random
from sklearn.model_selection import train_test_split
from keras.optimizers import SGD, Adam, RMSprop


Using TensorFlow backend.


In [2]:
import pandas as pd
data = pd.read_csv('MMdriving_log.csv', header = None)
data.columns = ["center_images","left_images","right_images","steering","brake","throttle","speed"]
steering = data['steering'] 
images = data['center_images']
len(data)

53416

In [5]:
X_train = np.copy(data['center_images'])
Y_train = np.copy(data['steering'])  

In [6]:
len(X_train)

53416

In [7]:
img_rows = 66
img_cols = 200
ch = 3

In [8]:
def load_image(imagepath):
    oriimage = cv2.imread(imagepath, 1)
    # do any preprocessing... resize, reshape, etc. here
    newimage = cv2.resize(oriimage, (200, 66))
    return newimage

In [9]:
batch_size = 32
nb_epoch = 3


In [10]:
"""
Steering angle prediction model
"""
import os
import argparse
import json
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Lambda, ELU
from keras.layers.convolutional import Convolution2D

In [11]:
# our image generator.
def batchgen(X, Y):
    while 1:
        for i in range(len(X)):
            y = Y[i]
            imagepath = X[i]
                       
            image = load_image(imagepath)
            y = np.array([[y]])
            image = image.reshape(1, img_rows, img_cols, ch)
            yield image, y

In [17]:
image.shape

NameError: name 'image' is not defined

In [12]:
#Formula is (image_size - (filter_size+1))/stride, for valid padding.

In [13]:
def get_model(time_len=1):
    # input: 80x160 images with 3 channels -> (3, 80, 160) tensors.
    row, col, ch = 66, 200, 3  # camera format

    model = Sequential()
    model.add(Lambda(lambda x: x/127.5 - 1.,
            input_shape=(row, col, ch),
            output_shape=(row, col, ch)))
    # this applies 24 convolution filters of size 5x5 each.
    model.add(Dropout(.2))
    model.add(Convolution2D(24, 5, 5, subsample=(2, 2), border_mode="same"))
    # Output of above Conv2D is (80-(5+1))/2 = 37 and (160-(5 + 1))/2 = 77
    # 24@37x77  Parameters from first conv = 24*5*5*3 = 
    # depth@convsize*convsize*ch = 1824
    model.add(ELU())
    model.add(Dropout(.2))    
    model.add(Convolution2D(36, 5, 5, subsample=(2, 2), border_mode="same"))
    # Output of above Conv2D is (37-(5+1))/2 = 15 and (77-(5 + 1))/2 = 35
    # 36@16x36  Paramenters from second conv = 24*36*5*5 = 21636
    model.add(ELU())
    model.add(Dropout(.2))
    model.add(Convolution2D(48, 5, 5, subsample=(2, 2), border_mode="same"))
    # Output of above Conv2D is (16-(5+1))/2 = 5 and (36-(5 + 1))/2 = 15
    # 48@5x15  Parameters 24*36*48*5*4
    model.add(ELU())
    model.add(Dropout(.2))
    model.add(Convolution2D(64, 3, 3, subsample=(1, 1), border_mode="same"))
    # Output of above Conv2D is (3-(3+1)) = 1 and (15-(3+ 1))= 10
    # 64@1x5  320 neurons
    model.add(ELU())
    model.add(Dropout(.2))
    model.add(Convolution2D(64, 3, 3, subsample=(1, 1), border_mode="same"))
    # Output of above Conv2D is (5-(3+1))/2 = 5 and (15-(3+ 1))/2 = 15
    # 64@1x5  320 neurons
    model.add(ELU())
    model.add(Flatten())
    model.add(Dropout(.2))
    model.add(ELU())
    model.add(Dense(32))
    model.add(Dropout(.5))
    model.add(ELU())
    model.add(Dense(10))
    model.add(Dropout(.5))
    model.add(ELU())
    model.add(Dense(1))
    model.add(Dropout(.5))
  
    model.compile(optimizer=Adam(), loss='mse')

    return model

In [14]:
model = get_model()

In [15]:
history = model.fit_generator(batchgen(X_train, Y_train), samples_per_epoch = 53416, nb_epoch = nb_epoch,
                    verbose=1, callbacks=[], validation_data=None,
                    class_weight=None, pickle_safe=False)

Epoch 1/5
53416/53416 [==============================] - 1592s - loss: 367.4448  
Epoch 2/5
53416/53416 [==============================] - 1673s - loss: 0.0252  
Epoch 3/5
53416/53416 [==============================] - 1798s - loss: 0.0252  
Epoch 4/5
53416/53416 [==============================] - 1783s - loss: 0.0252  
Epoch 5/5
53416/53416 [==============================] - 1716s - loss: 0.0252  


In [16]:
model.summary()

model.save_weights("model.h5", True)
with open('model.json', 'w') as outfile:
    json.dump(model.to_json(), outfile)
    
print("Saved model weights and configuration file.")

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 66, 200, 3)    0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 66, 200, 3)    0           lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 33, 100, 24)   1824        dropout_1[0][0]                  
____________________________________________________________________________________________________
elu_1 (ELU)                      (None, 33, 100, 24)   0           convolution2d_1[0][0]            
___________________________________________________________________________________________